In [1]:
from functools import reduce
import numpy as np
import re
from itertools import chain
from scipy import io
from scipy.signal import butter, lfilter
from scipy.linalg import eig
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score
import h5py
import csv

In [2]:
TRAIN_FILE = "train.h5"
TEST_FILE = "test.h5"
SAMPLING_RATE = 250

In [3]:
CHANNEL_NAMES = np.array(['T5', 'T3', 'F7', 'F3', 'C3', 'P3', 'Fp1', 'Fpz', 'A1', 'O1', 'Cz', 'Oz', 'Fz', 'Pz', 'O2', 'A2', 'Fp2', 'P4', 'C4', 'F4', 'F8', 'T4', 'T6', 'AUX'])
CHANNELS_TO_DISCARD = ['A1', 'A2', 'AUX']
EYE_CHANNEL = "Fp1"
CSP_FREQUENCES = [(6,10),(8,12),(10,14),(12,16),(14,18),(16,20),(18,22),(20,24),
                     (22,26),(24,28),(26,30),(28,32),(30,34),(32,36),(34,38)]

In [4]:
def butter_bandpass(lowcut, highcut, sampling_rate, order=5):
    """
    For a given order (5 by default), computes numerator (b)
    and denominator (a) polynomials
    of the IIR filter (bandpass between lowcut and highcut).
    """
    nyq_freq = sampling_rate*0.5
    low = lowcut/nyq_freq
    high = highcut/nyq_freq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_high_low_pass(lowcut, highcut, sampling_rate, order=5):
    """
    For a given order (5 by default), computes numerator (b_high, b_low)
    and denominator (a_high, a_low) polynomials of the IIR filters
    (highpass for lowcut, lowpass for highcut).
    """
    nyq_freq = sampling_rate*0.5
    lower_bound = lowcut/nyq_freq
    higher_bound = highcut/nyq_freq
    b_high, a_high = butter(order, lower_bound, btype='high')
    b_low, a_low = butter(order, higher_bound, btype='low')
    return b_high, a_high, b_low, a_low

def butter_bandpass_filter(data, lowcut, highcut, sampling_rate, order=5, how_to_filt = 'separately'):
    """
    Applies Butterworth bandpass filter (with a given order, 5 by default) to the data.
    If how_to_filt = 'separately', applies highpass than lowpass, and thus results
        in bandpass between lowcut and highcut.
    If how_to_filt == 'simultaneously', applies bandpass directly.
    """
    if how_to_filt == 'separately':
        b_high, a_high, b_low, a_low = butter_high_low_pass(lowcut, highcut, sampling_rate, order=order)
        y = lfilter(b_high, a_high, data)
        y = lfilter(b_low, a_low, y)
    elif how_to_filt == 'simultaneously':
        b, a = butter_bandpass(lowcut, highcut, sampling_rate, order=order)
        y = lfilter(b, a, data)
    return y

In [5]:
def remove_outliers(data, iter_numb):
    """
    Repetitively (specified by iter_numb) applies outliers removal to the data:
    deletes samples which power deviates more than 2.5 SDs from the mean.
    """
    data_pwr = np.sqrt(np.sum(data**2, 0))
    mask = np.ones(data.shape[1], dtype=np.bool)
    for i in range(iter_numb):
        X_mean = np.mean(data_pwr[mask])
        X_std = np.std(data_pwr[mask])
        mask &= np.abs(data_pwr - X_mean) < 2.5*np.abs(X_std)
        print('Samples left after outliers removal: %d' % mask.sum())
    return mask

def remove_eog_simple(data, eyechan, number_artefact_components=3):
    """
    Blinks and eye artefacts removal by singular value decomposition.
    
    Arguments:
    data: EEG data, numpy array [n_channels, n_samples]
    eyechan: channel nummer used for artefacts detection
    number_artefact_components: number of eigenvalues used - the number
       of components the artefacts are presumed to have
       
    Returns artefact-free EEG time series and the matrix of corresponding transformation
    """
    only_eye_chan = data[eyechan, :]
    exceed_mask = only_eye_chan > 3*np.mean(np.absolute(only_eye_chan))
    print('Number of samples identified as containing eye artefacts: %d' % np.sum(exceed_mask))
    U, S, V = np.linalg.svd(data[:, exceed_mask[0,:]], full_matrices=True)
    M_eog = np.eye(U.shape[0])-np.dot(U[:,0:number_artefact_components],U[:,0:number_artefact_components].T)
    
    return np.dot(M_eog, data), M_eog

In [6]:
def outer_n(n):
    return np.array(list(range(n))+list(range(-n,0)))

def whitener(C, rtol=1e-15):
    e, E = np.linalg.eigh(C)
    return reduce(np.dot, [E, np.diag(np.where(e > np.max(e) * rtol, e, np.inf)**-0.5), E.T])

def csp_base(C_a, C_b):
    P = whitener(C_a + C_b)
    P_C_b = reduce(np.dot, [P, C_b, P.T])
    _, _, B = np.linalg.svd((P_C_b))
    return np.dot(B, P.T)

def csp(C_a, C_b, m):
    W = csp_base(C_a, C_b)
    assert W.shape[1] >= 2*m
    return W[outer_n(m)]

In [7]:
def get_CSP_matr(data, states_labels, main_state, N_comp, other_state=None, mode='one_vs_all'):
    """
    Computes matrix of CSP (common spatial pattern) transform which maximizes the signal
    variance for one condition while minimizing the variance for the other condition.
    CSP method is based on the simultaneous diagonalization of the covariance matrices of both conditions.
    main_state is the label of a state for which variance will be maximized.
    if mode == 'one_vs_all', then the variance is minimized for all states other than main_state.
    if mode == 'pairwise', then the variance is minimized for a specified state (other_state).
    Returns matrix of eigenvectors corresponding to N_comp highest eigenvalues and
    N_comp lowest  eigenvalues.
    """
    A = data[:, (states_labels == main_state)[0,:]]
    if mode == 'one_vs_all':
        B = data[:, (states_labels != main_state)[0,:]]
    elif mode == 'pairwise':
        if other_state == None:
            print("Other state must be specified")
            return None
        else:
            B = data[:, (states_labels == other_state)[0,:]]
    
    C1 = np.cov(A)
    C2 = np.cov(B)
    
    return csp(C1,C2,N_comp)

In [8]:
def const_features(data,states_labels,states_codes,sr,feat_type,freq_ranges,how_to_filt,N_csp_comp,win,order=5,normalize=False):
    '''
    Filters data according to specified bands (in freq_range) and derives CSP transformations for each band.
    Type of CSP should be provided in feat_type: pairwise or one-vs-all (recommended).
    Number of CSP components should be provided in N_csp_comp (for a given N, N first and N last components will be used).
    Time interval for averaging is specified in win.
    If normalize=True, each data point is in [0,1].
    Returns array of transformed data, list of CSP transform matrices (in arrays), 
    and array of state codes for each of the final features (i.e., which state was first while this CSP projection was computed)
    '''
    final_data = np.zeros((1, data.shape[1]))
    all_CSPs = []
    where_states = []
    
    if feat_type == 'CSP_pairwise':
        for freq in freq_ranges:
            data_filt = butter_bandpass_filter(data, freq[0], freq[1], sr, order, how_to_filt)
            all_states_CSP = []
            for st in states_codes:
                for oth_st in np.array(states_codes)[np.array(states_codes)!=st]:
                    CSP_st = get_CSP_matr(data_filt, states_labels, st, N_csp_comp, other_state=oth_st, mode='pairwise')
                    all_states_CSP.append(np.dot(CSP_st, data_filt))
                    all_CSPs.append(CSP_st)
                    where_states.extend([st]*(N_csp_comp*2))
                data_transformed = np.vstack(all_states_CSP)**2
            final_data = np.vstack((final_data, data_transformed))
            
    elif feat_type == 'CSP_one_vs_all':
        for freq in freq_ranges:
            data_filt = butter_bandpass_filter(data, freq[0], freq[1], sr, order, how_to_filt)
            all_states_CSP = []
            for st in states_codes:
                CSP_st = get_CSP_matr(data_filt, states_labels, st, N_csp_comp, other_state=None, mode='one_vs_all')
                all_states_CSP.append(np.dot(CSP_st, data_filt))
                all_CSPs.append(CSP_st)
                where_states.extend([st]*(N_csp_comp*2))
            data_transformed = np.vstack(all_states_CSP)**2
            final_data = np.vstack((final_data, data_transformed))
            
    elif feat_type == 'no_filt_no_csp':
        final_data = np.vstack((final_data, data**2))

    final_data = final_data[1:,:]
    a_ma = 1
    b_ma = np.ones(win)/float(win)
    final_data = lfilter(b_ma, a_ma, final_data)
    if normalize:
        final_data = final_data/np.sum(final_data,0)[np.newaxis,:]
    print('Shape of data matrix: ', final_data.shape)
        
    return final_data, all_CSPs, np.array(where_states)

def filt_apply_CSPs(data, sr, freq_range, all_CSPs, how_to_filt, win, order=5, normalize=False, no_filt_no_csp=False):
    '''
    Filters data according to specified bands (in freq_range) and applies corresponding CSP transformations (in all_CSPs).
    Order in freq_range and all_CSPs must be the same.
    If normalize=True, each data point is in [0,1].
    '''
    if no_filt_no_csp == False:
        N_csp_per_freq = len(all_CSPs) // len(freq_range)
        all_CSPs_copy = list(all_CSPs)
        transformed_data = np.zeros((1, data.shape[1]))
        for fr_ind in range(len(freq_range)):
            filt_data = butter_bandpass_filter(data,freq_range[fr_ind][0],freq_range[fr_ind][1],sr,order,how_to_filt)
            for csp_ind in range(N_csp_per_freq):
                transformed_data = np.vstack((transformed_data, np.dot(all_CSPs_copy.pop(0), filt_data)))
        final_data = transformed_data[1:,:]**2
        
    elif no_filt_no_csp == True:
        final_data = data**2
        
    a_ma = 1
    b_ma = np.ones(win)/float(win)
    final_data = lfilter(b_ma, a_ma, final_data)
    if normalize:
        final_data = final_data/np.sum(final_data,0)[np.newaxis,:]
    return final_data

In [9]:
def filter_eeg(eeg_data, sampling_rate, channel_names, states_codes):
    # Prefilter eeg data
    eeg_data = butter_bandpass_filter(eeg_data, 0.5, 45, sampling_rate, order=5, how_to_filt='separately')
    
    # Remove empty channels
    # Detect constant (zero) channels
    channels_mask = np.ones(len(channel_names), dtype=np.bool)
    for channel in CHANNELS_TO_DISCARD:
        channels_mask &= (channel_names != channel)
    nozeros_mask = np.sum(eeg_data[:, :sampling_rate * 2], 1) != 0
    without_emp_mask = nozeros_mask & channels_mask
    
    # Remove constant (zero) channels and prespecified channels
    eeg_data = eeg_data[without_emp_mask, :]

    return eeg_data, without_emp_mask
    

def extract_features(eeg_data, states_labels, states_codes, sampling_rate):
    """
    Construct features: project eeg data on CSP components (separately for each of specified frequency bands)
    """
    N_CSP_comp = 3 # N first and N last; 2*N in total
    window_to_sampling_rate = 2
    win = sampling_rate // window_to_sampling_rate # Window for averaging: 0.5 sec
    eeg_data, all_CSPs, where_states = const_features(eeg_data,states_labels,states_codes,sampling_rate,'CSP_one_vs_all',
                                                        CSP_FREQUENCES,'separately',N_CSP_comp,win)
    
    return eeg_data[:, sampling_rate:], states_labels[:, sampling_rate:].ravel(), all_CSPs, win

In [10]:
numbergetter = re.compile(r"\d+$")
def get_tail_number(string):
    return int(numbergetter.findall(string)[0])

In [11]:
class BaselineModel(object):
    def __init__(self, classifier):
        self.classifier = classifier
    
    def fit(self, eeg, states_labels):
        states_codes = list(np.unique(states_labels))
        filtered_eeg, self.selected_channels = filter_eeg(
            eeg, SAMPLING_RATE, CHANNEL_NAMES, states_codes)

        # Remove outliers; remove artefacts (blinks, eye movements)
        eye_channel_index = (CHANNEL_NAMES[self.selected_channels] == EYE_CHANNEL)
        selected_items = remove_outliers(filtered_eeg, 7)
        filtered_eeg, self.M_eog = remove_eog_simple(filtered_eeg, eye_channel_index)

        extracted_features, labels, self.all_CSPs, self.averaging_window = extract_features(
            filtered_eeg[:, selected_items], states_labels[:, selected_items], states_codes, SAMPLING_RATE)

        self.classifier.fit(extracted_features.T, labels)
        
    def predict_proba(self, eeg):
        test_eeg = butter_bandpass_filter(
            eeg, 0.5, 45, SAMPLING_RATE, order=5, how_to_filt='separately')[self.selected_channels]
        test_eeg = self.M_eog.dot(test_eeg)
        test_features = filt_apply_CSPs(
            test_eeg, SAMPLING_RATE, CSP_FREQUENCES, self.all_CSPs, 'separately', 
            self.averaging_window, no_filt_no_csp=False)
        return self.classifier.predict_proba(test_features.T)

In [12]:
with h5py.File(TRAIN_FILE, 'r') as data_file:
    with h5py.File(TEST_FILE, 'r') as test_file:
        with open("baseline_submission.csv", "w") as submission_io:
            answers_writer = csv.writer(submission_io)
            answers_writer.writerow(
                ["subject_id", "chunk_id", "tick", "class_0_score", "class_1_score", "class_2_score"])
            for subject, subject_data in data_file.items():
                model = BaselineModel(RandomForestClassifier(n_jobs=-1))
                model.fit(subject_data['data'], subject_data['labels'])
                # The exercise of setting up the proper cross-validation we leave to the reader
                print("Subject %s; train ROC AUC: %f" % (subject, roc_auc_score(LabelBinarizer().fit_transform(
                            np.ravel(subject_data['labels'])),
                            model.predict_proba(subject_data['data']))))
                for chunk_id, chunk in test_file[subject].items():
                    chunk_prediction = model.predict_proba(chunk)
                    answers_writer.writerows([chain(
                            [get_tail_number(subject), get_tail_number(chunk_id), x[0]], x[1])
                                              for x in enumerate(chunk_prediction)])

Samples left after outliers removal: 100390
Samples left after outliers removal: 98884
Samples left after outliers removal: 95921
Samples left after outliers removal: 93717
Samples left after outliers removal: 92399
Samples left after outliers removal: 91637
Samples left after outliers removal: 91217
Number of samples identified as containing eye artifacts: 1001
Shape of data matrix:  (270, 91217)
Subject subject_0; train ROC AUC: 0.995772
Samples left after outliers removal: 53104
Samples left after outliers removal: 51461
Samples left after outliers removal: 50183
Samples left after outliers removal: 49349
Samples left after outliers removal: 48725
Samples left after outliers removal: 48263
Samples left after outliers removal: 47933
Number of samples identified as containing eye artifacts: 856
Shape of data matrix:  (270, 47933)
Subject subject_1; train ROC AUC: 0.977660
Samples left after outliers removal: 54905
Samples left after outliers removal: 54110
Samples left after outliers 